# Wykrywanie anomalii sieciowych

## Zadanie

Wytrenować autoenkoder na załaczonych dnaych pomiarowych z sieic AGH. Pamietać, że dane to szereg czasowy i autoenkoder trenujemy na oknie o romiarze $w$. Zbudowanie PCA tez będzie unznane, ale na najnizsza ocene umożliwiająca zaliczenie.

Uwagi

- Główne etapy analizy powinny byc rodzieleone na komórki
- Rozwiązanie przysyłamy w formie notatnika z wynikami
- Należy skomentować podjete akcje

## Dane grupy

- Heorhi Kupryianau
- Mikita Vostry

### Wczytujemy dane

In [252]:
import pandas as pd

df = pd.read_csv('../data/stats_clean.csv')

df.head()

,timestamp,b6-ucirtr,ucirtr-b6,b6-b1rtr,b1rtr-b6,b1-ftjrtr,ftjrtr-b1,cyfronet-ucirtr,ucirtr-cyfronet,ftj-b6rtr,...,ftj-ucirtr,ucirtr-ftj,uci-ftjrtr,ftjrtr-uci,uci-b1rtr,b1rtr-uci,ftj-b1rtr,b1rtr-ftj,uci-b6rtr,b6rtr-uci
0,2020-12-13 15:25:03,4063963.0,2099589.0,1728848.0,52632.0,2352.0,16766.0,10425570.0,39459770.0,82891.0,...,5046928.0,1831919.0,1845690.0,5044319.0,8395469.0,2898609.0,16534.0,2314.0,2104685.0,4062125.0
1,2020-12-13 15:30:03,4115472.0,2033890.0,1675340.0,46513.0,2468.0,17191.0,11739436.0,37856957.0,29267.0,...,5468130.0,1557696.0,1562774.0,5506855.0,8389581.0,3139075.0,17260.0,2481.0,2033834.0,4113483.0
2,2020-12-13 15:35:03,4460894.0,1967785.0,1717643.0,46969.0,2572.0,17776.0,14880181.0,22968938.0,19467.0,...,4910226.0,1682946.0,1674507.0,4880848.0,7446018.0,2968990.0,17838.0,2586.0,1960732.0,4464048.0
3,2020-12-13 15:40:03,4180486.0,2008196.0,1616511.0,47048.0,2451.0,16932.0,13961237.0,23170350.0,24386.0,...,5883441.0,2946501.0,2943126.0,5899514.0,7788634.0,4172991.0,16892.0,2439.0,2014920.0,4178625.0
4,2020-12-13 15:45:03,3744176.0,3335874.0,1746441.0,56408.0,2834.0,17673.0,14983896.0,24006404.0,29554.0,...,4670566.0,2467885.0,2503704.0,4727394.0,7436740.0,3538008.0,17450.0,2797.0,3334540.0,3739133.0


### Usuwamy NaN wartości

In [253]:
l = len(df)

df.dropna(inplace=True)

print(f"length before: {l}\nlength after: {len(clean_df)}")

length before: 103194
length after: 102850


### Dzielimy dane na zbiór trenujący i testowy

In [254]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_n, test_size=0.25)

print(f"length: {len(df_train)}")
df_train.head()

length: 77137


,timestamp,b6-ucirtr,ucirtr-b6,b6-b1rtr,b1rtr-b6,b1-ftjrtr,ftjrtr-b1,cyfronet-ucirtr,ucirtr-cyfronet,ftj-b6rtr,...,ftj-ucirtr,ucirtr-ftj,uci-ftjrtr,ftjrtr-uci,uci-b1rtr,b1rtr-uci,ftj-b1rtr,b1rtr-ftj,uci-b6rtr,b6rtr-uci
25644,1.0,5.166692e-12,4.867424e-12,1.777643e-14,1.780193e-13,1.600058e-15,1.434234e-14,1.811889e-11,1.651647e-11,1.555058e-14,...,3.086719e-12,3.222228e-12,3.254811e-12,3.112642e-12,1.164525e-11,4.698778e-12,1.423897e-14,1.583965e-15,4.867491e-12,5.165847e-12
39832,1.0,8.104578e-13,1.700330e-12,1.103824e-15,3.612123e-15,1.461271e-15,1.082649e-14,2.650639e-11,3.505080e-12,7.636638e-15,...,1.748754e-12,3.692379e-13,3.654307e-13,1.727685e-12,3.054845e-12,6.805084e-13,1.087341e-14,1.469914e-15,1.713670e-12,8.517104e-13
38369,1.0,1.385855e-12,1.572313e-12,2.876094e-14,8.785255e-14,7.806673e-15,3.480416e-13,1.074000e-11,6.573912e-12,9.204735e-15,...,2.527951e-12,7.888420e-13,7.834449e-13,2.509554e-12,3.284735e-12,1.088134e-12,3.502641e-13,7.862867e-15,1.576119e-12,1.385336e-12
45510,1.0,1.607698e-12,2.736814e-12,1.041920e-14,7.375582e-14,1.600344e-15,1.549528e-14,1.363931e-11,1.426866e-11,1.502732e-13,...,3.249212e-12,1.977739e-12,1.967686e-12,3.229861e-12,8.145660e-12,3.953943e-12,1.562972e-14,1.613295e-15,2.734785e-12,1.607656e-12
36246,1.0,2.391807e-12,1.501356e-12,1.587648e-15,3.665798e-14,7.471211e-15,3.155527e-13,4.733270e-12,7.965646e-12,4.679484e-14,...,2.928787e-12,1.234511e-12,1.173553e-12,2.903108e-12,4.816326e-12,1.402930e-12,3.176148e-13,7.523103e-15,1.495672e-12,2.358032e-12


### Standaryzacja danych
Zbiór zawiera wartość nieliczbową 'timestamp'

In [255]:
from sklearn.preprocessing import StandardScaler

df_train = df_train.set_index('timestamp')
df_test = df_test.set_index('timestamp')

scaler = StandardScaler()

train = scaler.fit_transform(df_train.values)
test = scaler.transform(df_test.values)

df_train = pd.DataFrame(train, index=df_train.index, columns=df_train.columns)
df_test = pd.DataFrame(test, index=df_test.index, columns=df_test.columns)

df_train.head()

,b6-ucirtr,ucirtr-b6,b6-b1rtr,b1rtr-b6,b1-ftjrtr,ftjrtr-b1,cyfronet-ucirtr,ucirtr-cyfronet,ftj-b6rtr,b6rtr-ftj,...,ftj-ucirtr,ucirtr-ftj,uci-ftjrtr,ftjrtr-uci,uci-b1rtr,b1rtr-uci,ftj-b1rtr,b1rtr-ftj,uci-b6rtr,b6rtr-uci
timestamp,,,,,,,,,,,,,,,,,,,,,
1.0,0.622782,0.057241,-0.335121,0.023992,-0.116115,-0.161280,-0.094898,0.617826,-0.100467,-0.074369,...,0.060101,-0.373267,-0.368980,0.069384,0.328539,0.977742,-0.161840,-0.117849,0.057248,0.622639
1.0,-0.354871,-0.515597,-0.366765,-0.207744,-0.122849,-0.181654,0.312318,-0.830832,-0.124723,-0.043229,...,-0.421401,-0.746169,-0.746585,-0.429004,-0.648717,-0.643006,-0.181334,-0.123432,-0.513161,-0.345621
1.0,-0.225736,-0.538751,-0.314273,-0.095813,0.185039,1.772443,-0.453145,-0.489155,-0.119917,-0.083509,...,-0.140987,-0.691324,-0.691956,-0.147642,-0.622564,-0.478592,1.784450,0.189470,-0.538039,-0.225854
1.0,-0.175949,-0.328126,-0.349085,-0.114543,-0.116101,-0.154599,-0.312383,0.367560,0.312445,1.011241,...,0.118578,-0.535929,-0.537191,0.111567,-0.069579,0.677317,-0.153785,-0.116414,-0.328479,-0.175957
1.0,0.000026,-0.551585,-0.365847,-0.163836,0.168762,1.584176,-0.744774,-0.334202,-0.004706,-0.081327,...,0.003265,-0.633073,-0.640974,-0.006018,-0.448328,-0.351621,1.595342,0.172841,-0.552589,-0.007544


### Tworzymy Encoder

In [263]:
import tensorflow as tf

input_dim = df_train.shape[1]

model = tf.keras.models.Sequential()
model.add(
tf.keras.layers.Dense(
        units=25, 
        input_dim=input_dim,
        activation=tf.nn.relu
    ))
model.add(
    tf.keras.layers.Dense(
        units=15,
        activation=tf.nn.relu
    ))

### Tworzymy Latent Space

In [264]:
model.add(
    tf.keras.layers.Dense(
        units=8,
        activation=tf.nn.relu
    ))

### Tworzymy Decoder

In [265]:
model.add(
    tf.keras.layers.Dense(
        units=15,
        activation=tf.nn.relu
    ))
model.add(
    tf.keras.layers.Dense(
        units=25,
        activation=tf.nn.relu
    ))
model.add(
    tf.keras.layers.Dense(
        units=input_dim,
    ))

### Kompilujemy model

In [266]:
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_124 (Dense)           (None, 25)                775       
                                                                 
 dense_125 (Dense)           (None, 15)                390       
                                                                 
 dense_126 (Dense)           (None, 8)                 128       
                                                                 
 dense_127 (Dense)           (None, 15)                135       
                                                                 
 dense_128 (Dense)           (None, 25)                400       
                                                                 
 dense_129 (Dense)           (None, 30)                780       
                                                                 
Total params: 2,608
Trainable params: 2,608
Non-train

### Uruchamiamy model

In [267]:
model.fit(df_train, df_train, epochs=32)

Epoch 1/32
2411/2411 [==============================] - 3s 1ms/step - loss: 0.4739
Epoch 2/32
2411/2411 [==============================] - 3s 1ms/step - loss: 0.2232
Epoch 3/32
2411/2411 [==============================] - 3s 1ms/step - loss: 0.1647
Epoch 4/32
2411/2411 [==============================] - 3s 1ms/step - loss: 0.1424
Epoch 5/32
2411/2411 [==============================] - 3s 1ms/step - loss: 0.1305
Epoch 6/32
2411/2411 [==============================] - 3s 1ms/step - loss: 0.1228
Epoch 7/32
2411/2411 [==============================] - 3s 1ms/step - loss: 0.1167
Epoch 8/32
2411/2411 [==============================] - 3s 1ms/step - loss: 0.1162
Epoch 9/32
2411/2411 [==============================] - 3s 1ms/step - loss: 0.0899
Epoch 10/32
2411/2411 [==============================] - 3s 1ms/step - loss: 0.0769
Epoch 11/32
2411/2411 [==============================] - 3s 1ms/step - loss: 0.0784
Epoch 12/32
2411/2411 [==============================] - 3s 1ms/step - loss: 0.0771
E

### Uruchamiamy model dla danych testowych

In [268]:
p = model.predict(df_test)
pd.DataFrame(p).head()

804/804 [==============================] - 1s 980us/step


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-0.417448,-0.594767,1.596100,-0.119484,-0.212061,-0.156555,-0.546087,-0.907197,-0.096555,-0.056888,...,-0.512422,-0.679006,-0.658599,-0.511798,-0.362015,-0.759897,-0.154693,-0.224551,-0.601664,-0.403869
1,1.617532,0.127713,-0.152560,2.256194,-0.179386,-0.063533,0.652404,1.764296,0.027380,0.054143,...,0.355435,-0.168105,-0.154813,0.347592,0.943461,1.077037,-0.058608,-0.156338,0.124254,1.622305
2,0.059848,0.617559,-0.303426,-0.150790,-0.136587,-0.139637,0.021810,0.078361,-0.037366,-0.041008,...,-0.345371,-0.420779,-0.416274,-0.350147,0.070225,-0.030989,-0.133493,-0.117325,0.619311,0.061539
3,-0.178013,-0.703297,1.262607,-0.176141,-0.148238,-0.186473,-0.459100,-0.009457,-0.041971,-0.100276,...,-0.288074,0.420435,0.445447,-0.301615,-0.671030,-0.397211,-0.200096,-0.119766,-0.717512,-0.173201
4,-0.462081,-0.658943,-0.373325,-0.187485,-0.184840,-0.157304,-0.414952,-0.681221,-0.053240,0.023576,...,-0.548098,0.371211,0.380893,-0.553308,-0.639452,-0.606044,-0.158376,-0.179403,-0.664251,-0.455744


### Obliczamy błąd dopasowania

In [270]:
from sklearn import metrics

ms_error = metrics.mean_squared_error(p, df_test)
ms_error

0.15768669039722472